In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# def atoi(text):
#     return int(text) if text.isdigit() else text

# def natural_keys(text):
#     return [ atoi(c) for c in re.split('(\d+)', text) ]

# def load_flower_data():
#     # grab the list of images that we'll be describing
#     print("[INFO] handling images...")
#     TRAIN_ORIGINAL_DIR = '../train/'
#     TRAIN_SUB_DIR = '../subsample/'
#     TRAIN_GAN = '../../image_gan/'
#     TEST_DIR = '../../test/'

#     # use this for full dataset
#     train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
#     test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
#     train_images = train_images_gan
    
#     train_images.sort(key=natural_keys)
#     test_images.sort(key=natural_keys)

#     # initialize the features matrix and labels list
#     trainImage = []
#     trainLabels = []
#     testImage = []
#     testLabels = []

#     # loop over the input images
#     for (i, imagePath) in enumerate(train_images):
#         # extract the class label
#         # get the labels from the name of the images by extract the string before "_"
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # read and resize image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         trainImage.append(img)
#         trainLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
#       # loop over the input images
#     for (i, imagePath) in enumerate(test_images):
#         # extract the class label
#         # our images were named as labels.image_number.format
#         # get the labels from the name of the images by extract the string before "."
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # extract CNN features in the image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         testImage.append(img)
#         testLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


#     trainImage = np.array(trainImage,dtype = float32)
#     trainLabels = np.array(trainLabels)
#     testImage = np.array(testImage,dtype = float32)
#     testLabels = np.array(testLabels)
#     print (trainImage.shape)
    
#     trainImage = trainImage.astype(np.float32) / 255
#     testImage = testImage.astype(np.float32) / 255
    
#     le = preprocessing.LabelEncoder()
#     le.fit(trainLabels)
#     list(le.classes_)
#     trainLabels = le.transform(trainLabels) 
#     testLabels = le.transform(testLabels) 
    
#     return trainImage, trainLabels, testImage, testLabels

In [3]:
# trainImage, trainLabels, testImage, testLabels = load_flower_data()

# nb_classes = 2

# # Convert class vectors to binary class matrices.
# trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
# print (trainLabels)
# testLabels = keras.utils.to_categorical(testLabels, nb_classes)
# print (testLabels)
# print (testLabels.shape)

# np.save('../trainImage.npy', trainImage)
# np.save('../trainLabels.npy', trainLabels)
# np.save('../testImage.npy', testImage)
# np.save('../testLabels.npy', testLabels)

# print("[INFO] trainImage matrix: {:.2f}MB".format(
#     (trainImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] trainLabels matrix: {:.4f}MB".format(
#     (trainLabels.nbytes) / (1024 * 1000.0)))
# print("[INFO] testImage matrix: {:.2f}MB".format(
#     (testImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] testLabels matrix: {:.4f}MB".format(
#     (testLabels.nbytes) / (1024 * 1000.0)))


In [4]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [128] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # train   
        self.global_step = tf.Variable(0, trainable=False)
        self.initial_lr = 0.001 #初始学习率
        self.decay_steps = 5
        self.decay_rate = 0.9
        self.lr = tf.train.exponential_decay( self.initial_lr,
                                     global_step=self.global_step,
                                     decay_steps=self.decay_steps,
                                     decay_rate=self.decay_rate)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # name for checkpoint
        self.model_name = 'CNN_C%d_D%d_Kernel(%d,%d)_%d_lrdecay0.0001' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))    
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y= np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y= np.load('../testLabels.npy')
        
        print ("Training:",self.train_x.shape[0])
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 

            #输入,卷积核为3*3 输出维度为32
            net = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1'               # 命名用于获取变量
                                    )
            print("CNN:",net.get_shape())

            if self.batch_normalization_control:
                net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            net = tf.layers.max_pooling2d(inputs = net,
                                          pool_size = self.pool_size,
                                          strides = (2, 2),
                                          padding='same',
                                          name='pool_conv_' + str(1)
                                         )
            print("CNN:",net.get_shape())
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )

                print("CNN:",net.get_shape())

            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
            
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'_train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status            
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, 
                                                                                    self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                rand_index = np.random.choice(len(self.train_x), size=batch_size)
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                   self.y: batch_y_tes,
                                                                   self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
            
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
                
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append(test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '_train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '_train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
    
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")
        
    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')
                          
        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [5]:
gan_type = 'CNN'
dataset = '4_Flowers'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
    #CNN.test(epoch)
        
sess.close()
        

# lrdecay
# Avg per epoch ptime: 9.94, total 30 epochs ptime: 43.01
#  [*] Training finished!
#  [*] Best Epoch:  18 , Accuracy:  0.7799999713897705
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_C3_D1_Kernel(3,3)_128_lrdecay0.0001/CNN_C3_D1_Kernel(3,3)_128_lrdecay0.0001-18
#  [*] Finished testing Best Epoch: 18 , accuracy:  0.7799999713897705 !

# Avg per epoch ptime: 10.12, total 50 epochs ptime: 532.53
#  [*] Training finished!
#  [*] Best Epoch:  8 , Accuracy:  0.8500000238418579
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001/CNN_C5_D1_Kernel(3,3)_128_lrdecay0.0001-8
#  [*] Finished testing Best Epoch: 8 , accuracy:  0.8500000238418579 !

Training: 6000
CNN:x (100, 128, 128, 3)
CNN: (100, 128, 128, 32)
CNN: (100, 64, 64, 32)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN: (100, 128, 128, 32)
CNN: (100, 64, 64, 32)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/beta:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/gamma:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_2/kernel:0 (float32_ref 3x3x32x64) [184

Epoch: [ 2] [  44/  60] time: 25.1189, loss: 0.08154096
Epoch: [ 2] [  45/  60] time: 25.2663, loss: 0.04909135
Epoch: [ 2] [  46/  60] time: 25.4197, loss: 0.02871477
Epoch: [ 2] [  47/  60] time: 25.5666, loss: 0.12036028
Epoch: [ 2] [  48/  60] time: 25.7200, loss: 0.06061158
Epoch: [ 2] [  49/  60] time: 25.8695, loss: 0.04818199
Epoch: [ 2] [  50/  60] time: 26.0254, loss: 0.02859569
Epoch: [ 2] [  51/  60] time: 26.1736, loss: 0.09782997
Epoch: [ 2] [  52/  60] time: 26.3260, loss: 0.03743186
Epoch: [ 2] [  53/  60] time: 26.4796, loss: 0.04562238
Epoch: [ 2] [  54/  60] time: 26.6324, loss: 0.08662450
Epoch: [ 2] [  55/  60] time: 26.7792, loss: 0.13185869
Epoch: [ 2] [  56/  60] time: 26.9321, loss: 0.08442102
Epoch: [ 2] [  57/  60] time: 27.0799, loss: 0.06144232
Epoch: [ 2] [  58/  60] time: 27.2354, loss: 0.07512767
Epoch: [ 2] [  59/  60] time: 27.3883, loss: 0.15245771
[2/50] - ptime: 13.2178 loss: 0.07688704 acc: 0.77000 lr: 0.00095873
Epoch: [ 3] [   0/  60] time: 29.28

Epoch: [ 5] [   8/  60] time: 51.8802, loss: 0.06007268
Epoch: [ 5] [   9/  60] time: 52.0295, loss: 0.01023073
Epoch: [ 5] [  10/  60] time: 52.1836, loss: 0.01267240
Epoch: [ 5] [  11/  60] time: 52.3353, loss: 0.01502322
Epoch: [ 5] [  12/  60] time: 52.4944, loss: 0.03775498
Epoch: [ 5] [  13/  60] time: 52.6579, loss: 0.00927843
Epoch: [ 5] [  14/  60] time: 52.8217, loss: 0.03170023
Epoch: [ 5] [  15/  60] time: 52.9836, loss: 0.02242054
Epoch: [ 5] [  16/  60] time: 53.1408, loss: 0.05509279
Epoch: [ 5] [  17/  60] time: 53.3026, loss: 0.02860595
Epoch: [ 5] [  18/  60] time: 53.4628, loss: 0.02106811
Epoch: [ 5] [  19/  60] time: 53.6248, loss: 0.04815090
Epoch: [ 5] [  20/  60] time: 53.7934, loss: 0.03049426
Epoch: [ 5] [  21/  60] time: 53.9531, loss: 0.06892401
Epoch: [ 5] [  22/  60] time: 54.1211, loss: 0.02421471
Epoch: [ 5] [  23/  60] time: 54.2849, loss: 0.00418713
Epoch: [ 5] [  24/  60] time: 54.4482, loss: 0.02310020
Epoch: [ 5] [  25/  60] time: 54.6087, loss: 0.0

Epoch: [ 7] [  32/  60] time: 76.8489, loss: 0.01198537
Epoch: [ 7] [  33/  60] time: 76.9969, loss: 0.00577732
Epoch: [ 7] [  34/  60] time: 77.1508, loss: 0.05844296
Epoch: [ 7] [  35/  60] time: 77.2990, loss: 0.01443684
Epoch: [ 7] [  36/  60] time: 77.4527, loss: 0.02359476
Epoch: [ 7] [  37/  60] time: 77.6056, loss: 0.05179471
Epoch: [ 7] [  38/  60] time: 77.7626, loss: 0.01656779
Epoch: [ 7] [  39/  60] time: 77.9096, loss: 0.01627077
Epoch: [ 7] [  40/  60] time: 78.0648, loss: 0.00376149
Epoch: [ 7] [  41/  60] time: 78.2158, loss: 0.01037143
Epoch: [ 7] [  42/  60] time: 78.3701, loss: 0.01480580
Epoch: [ 7] [  43/  60] time: 78.5175, loss: 0.04464518
Epoch: [ 7] [  44/  60] time: 78.6725, loss: 0.01737102
Epoch: [ 7] [  45/  60] time: 78.8217, loss: 0.00713764
Epoch: [ 7] [  46/  60] time: 78.9783, loss: 0.02063659
Epoch: [ 7] [  47/  60] time: 79.1313, loss: 0.01161821
Epoch: [ 7] [  48/  60] time: 79.2863, loss: 0.01845021
Epoch: [ 7] [  49/  60] time: 79.4338, loss: 0.0

Epoch: [ 9] [  56/  60] time: 101.6005, loss: 0.00498534
Epoch: [ 9] [  57/  60] time: 101.7603, loss: 0.00700500
Epoch: [ 9] [  58/  60] time: 101.9210, loss: 0.01654106
Epoch: [ 9] [  59/  60] time: 102.0821, loss: 0.00247515
[9/50] - ptime: 9.9404 loss: 0.02157638 acc: 0.68000 lr: 0.00082725
Epoch: [10] [   0/  60] time: 103.6312, loss: 0.01592394
Epoch: [10] [   1/  60] time: 103.7846, loss: 0.00536506
Epoch: [10] [   2/  60] time: 103.9347, loss: 0.01072802
Epoch: [10] [   3/  60] time: 104.0872, loss: 0.00827209
Epoch: [10] [   4/  60] time: 104.2426, loss: 0.01750010
Epoch: [10] [   5/  60] time: 104.3961, loss: 0.00070631
Epoch: [10] [   6/  60] time: 104.5512, loss: 0.01566831
Epoch: [10] [   7/  60] time: 104.7004, loss: 0.00971365
Epoch: [10] [   8/  60] time: 104.8496, loss: 0.00085186
Epoch: [10] [   9/  60] time: 105.0055, loss: 0.00981448
Epoch: [10] [  10/  60] time: 105.1689, loss: 0.00303925
Epoch: [10] [  11/  60] time: 105.3178, loss: 0.00135682
Epoch: [10] [  12/  

...

Epoch: [40] [  42/  60] time: 425.2537, loss: 0.00014518
Epoch: [40] [  43/  60] time: 425.4021, loss: 0.00004239
Epoch: [40] [  44/  60] time: 425.5559, loss: 0.00008623
Epoch: [40] [  45/  60] time: 425.7022, loss: 0.00067917
Epoch: [40] [  46/  60] time: 425.8566, loss: 0.00012710
Epoch: [40] [  47/  60] time: 426.0090, loss: 0.00022768
Epoch: [40] [  48/  60] time: 426.1659, loss: 0.00002149
Epoch: [40] [  49/  60] time: 426.3167, loss: 0.00001958
Epoch: [40] [  50/  60] time: 426.4720, loss: 0.00026284
Epoch: [40] [  51/  60] time: 426.6198, loss: 0.00019894
Epoch: [40] [  52/  60] time: 426.7739, loss: 0.00001436
Epoch: [40] [  53/  60] time: 426.9216, loss: 0.00156976
Epoch: [40] [  54/  60] time: 427.0791, loss: 0.00014897
Epoch: [40] [  55/  60] time: 427.2264, loss: 0.00002159
Epoch: [40] [  56/  60] time: 427.3835, loss: 0.00018028
Epoch: [40] [  57/  60] time: 427.5327, loss: 0.00007566
Epoch: [40] [  58/  60] time: 427.6889, loss: 0.00004412
Epoch: [40] [  59/  60] time: 4

Epoch: [43] [   4/  60] time: 450.5962, loss: 0.00008532
Epoch: [43] [   5/  60] time: 450.7439, loss: 0.00012930
Epoch: [43] [   6/  60] time: 450.8992, loss: 0.00005789
Epoch: [43] [   7/  60] time: 451.0497, loss: 0.00021773
Epoch: [43] [   8/  60] time: 451.2069, loss: 0.00002332
Epoch: [43] [   9/  60] time: 451.3577, loss: 0.00002601
Epoch: [43] [  10/  60] time: 451.5148, loss: 0.00356625
Epoch: [43] [  11/  60] time: 451.6671, loss: 0.00028666
Epoch: [43] [  12/  60] time: 451.8221, loss: 0.00007013
Epoch: [43] [  13/  60] time: 451.9703, loss: 0.00060082
Epoch: [43] [  14/  60] time: 452.1271, loss: 0.00002231
Epoch: [43] [  15/  60] time: 452.2745, loss: 0.00028383
Epoch: [43] [  16/  60] time: 452.4289, loss: 0.00029828
Epoch: [43] [  17/  60] time: 452.5772, loss: 0.00076535
Epoch: [43] [  18/  60] time: 452.7334, loss: 0.00001371
Epoch: [43] [  19/  60] time: 452.8833, loss: 0.00019882
Epoch: [43] [  20/  60] time: 453.0385, loss: 0.00003078
Epoch: [43] [  21/  60] time: 4

Epoch: [45] [  26/  60] time: 474.7051, loss: 0.00009268
Epoch: [45] [  27/  60] time: 474.8532, loss: 0.00025586
Epoch: [45] [  28/  60] time: 475.0083, loss: 0.00006907
Epoch: [45] [  29/  60] time: 475.1609, loss: 0.00032032
Epoch: [45] [  30/  60] time: 475.3182, loss: 0.00076538
Epoch: [45] [  31/  60] time: 475.4697, loss: 0.00003937
Epoch: [45] [  32/  60] time: 475.6245, loss: 0.00232362
Epoch: [45] [  33/  60] time: 475.7714, loss: 0.00005402
Epoch: [45] [  34/  60] time: 475.9245, loss: 0.00035342
Epoch: [45] [  35/  60] time: 476.0739, loss: 0.00014573
Epoch: [45] [  36/  60] time: 476.2292, loss: 0.01816482
Epoch: [45] [  37/  60] time: 476.3779, loss: 0.00320815
Epoch: [45] [  38/  60] time: 476.5349, loss: 0.00030680
Epoch: [45] [  39/  60] time: 476.6846, loss: 0.00022505
Epoch: [45] [  40/  60] time: 476.8400, loss: 0.00029205
Epoch: [45] [  41/  60] time: 476.9928, loss: 0.00053058
Epoch: [45] [  42/  60] time: 477.1489, loss: 0.00087159
Epoch: [45] [  43/  60] time: 4

Epoch: [47] [  48/  60] time: 498.8972, loss: 0.00003006
Epoch: [47] [  49/  60] time: 499.0488, loss: 0.00002353
Epoch: [47] [  50/  60] time: 499.2058, loss: 0.00036633
Epoch: [47] [  51/  60] time: 499.3568, loss: 0.00010588
Epoch: [47] [  52/  60] time: 499.5102, loss: 0.00001112
Epoch: [47] [  53/  60] time: 499.6632, loss: 0.00011015
Epoch: [47] [  54/  60] time: 499.8176, loss: 0.00001838
Epoch: [47] [  55/  60] time: 499.9660, loss: 0.00002143
Epoch: [47] [  56/  60] time: 500.1217, loss: 0.00049520
Epoch: [47] [  57/  60] time: 500.2719, loss: 0.00000507
Epoch: [47] [  58/  60] time: 500.4275, loss: 0.00024849
Epoch: [47] [  59/  60] time: 500.5770, loss: 0.00004469
[47/50] - ptime: 9.9241 loss: 0.00033227 acc: 0.67000 lr: 0.00037143
Epoch: [48] [   0/  60] time: 501.9120, loss: 0.00003155
Epoch: [48] [   1/  60] time: 502.0612, loss: 0.00026545
Epoch: [48] [   2/  60] time: 502.2176, loss: 0.00042014
Epoch: [48] [   3/  60] time: 502.3685, loss: 0.00007368
Epoch: [48] [   4/ 

Epoch: [50] [  10/  60] time: 524.3511, loss: 0.00099000
Epoch: [50] [  11/  60] time: 524.4994, loss: 0.00000482
Epoch: [50] [  12/  60] time: 524.6561, loss: 0.00011624
Epoch: [50] [  13/  60] time: 524.8061, loss: 0.00000260
Epoch: [50] [  14/  60] time: 524.9590, loss: 0.00001692
Epoch: [50] [  15/  60] time: 525.1107, loss: 0.00001521
Epoch: [50] [  16/  60] time: 525.2658, loss: 0.00000804
Epoch: [50] [  17/  60] time: 525.4142, loss: 0.00001207
Epoch: [50] [  18/  60] time: 525.5691, loss: 0.00007041
Epoch: [50] [  19/  60] time: 525.7174, loss: 0.00003705
Epoch: [50] [  20/  60] time: 525.8719, loss: 0.00004809
Epoch: [50] [  21/  60] time: 526.0233, loss: 0.00043253
Epoch: [50] [  22/  60] time: 526.1785, loss: 0.00000832
Epoch: [50] [  23/  60] time: 526.3302, loss: 0.00010589
Epoch: [50] [  24/  60] time: 526.4848, loss: 0.00036128
Epoch: [50] [  25/  60] time: 526.6328, loss: 0.00000657
Epoch: [50] [  26/  60] time: 526.7875, loss: 0.00002421
Epoch: [50] [  27/  60] time: 5

In [6]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放